In [1]:
import pandas as pd

    В этом руководстве используется набор данных Titanic, сохраненный в формате CSV. Данные состоят из следующих столбцов данных:
    
    * PassengerId: идентификатор каждого пассажира.
    * Survived: указывает, выжил ли пассажир. `0` за да и `1` за нет.
    * Pclass: один из 3 классов билетов: Class `1`, Class `2` и `Class 3`.
    * Name: Имя пассажира.
    * Sex: Пол пассажира.
    * Age: Возраст пассажира в годах.
    * SibSp: Количество братьев и сестер или супругов на борту.
    * Parch: Количество родителей или детей на борту.
    * Ticket: номер билета пассажира.
    * Fare: указание тарифа.
    * Cabin: Номер кабины пассажира.
    * Embarked: Порт посадки.

In [2]:
titanic = pd.read_csv("data/titanic.csv")

titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Как рассчитать сводную статистику

## Агрегированная статистика

![06_aggregate](https://pandas.pydata.org/docs/_images/06_aggregate.svg)

___

⸮ Каков средний возраст пассажиров Титаника?

In [3]:
titanic["Age"].mean()

29.69911764705882

Доступны различные статистические данные, которые можно применять к столбцам с числовыми данными. Операции в целом исключают отсутствующие данные и по умолчанию работают со строками.

![06_reduction](https://pandas.pydata.org/docs/_images/06_reduction.svg)
___

⸮ Каков средний возраст и стоимость билета пассажиров Титаника?

In [4]:
titanic[["Age", "Fare"]].median()

Age     28.0000
Fare    14.4542
dtype: float64

Статистика, применяемая к нескольким столбцам `DataFrame` (выбор двух столбцов возвращает `DataFrame`, см. руководство по подмножеству данных), рассчитывается для каждого числового столбца.

Агрегирующая статистика может быть рассчитана для нескольких столбцов одновременно. Помните функцию `describe` из первого урока?

In [5]:
titanic[["Age", "Fare"]].describe()

,Age,Fare
count,714.000000,891.000000
mean,29.699118,32.204208
std,14.526497,49.693429
min,0.420000,0.000000
25%,20.125000,7.910400
50%,28.000000,14.454200
75%,38.000000,31.000000
max,80.000000,512.329200


Вместо предопределенной статистики определенные комбинации агрегированной статистики для заданных столбцов можно определить с помощью метода `DataFrame.agg()`:

In [6]:
titanic.agg(
    {
        "Age": ["min", "max", "median", "skew"],
        "Fare": ["min", "max", "median", "mean"],
    }
)

,Age,Fare
min,0.420000,0.000000
max,80.000000,512.329200
median,28.000000,14.454200
skew,0.389108,NaN
mean,NaN,32.204208


## Агрегированная статистика, сгруппированная по категориям

![06_groupby](https://pandas.pydata.org/docs/_images/06_groupby.svg)

___

⸮ Каков средний возраст мужчин и женщин пассажиров Титаника?

In [7]:
titanic[["Sex", "Age"]].groupby("Sex").mean()

,Age
Sex,
female,27.915709
male,30.726645


Поскольку нас интересует средний возраст для каждого пола, сначала делается выборка по этим двум столбцам: `titanic[["Sex", "Age"]]`. Затем метод `groupby()` применяется к столбцу `Sex`, чтобы создать группу для каждой категории. Рассчитывается и возвращается средний возраст для каждого пола.

Расчет заданной статистики (например, среднего возраста) для каждой категории в столбце (например, мужчина/женщина в столбце «Пол») является распространенным шаблоном. Для поддержки этого типа операций используется метод `groupby`. Это соответствует более общему шаблону разделения-применения-объединения:

* **Разделите** данные на группы
* **Применение** функции к каждой группе независимо
* **Объедините** результаты в структуру данных

Шаги применения и объединения обычно выполняются вместе.

В предыдущем примере мы сначала явно выбрали 2 столбца. Если нет, метод `mean` применяется к каждому столбцу, содержащему числовые столбцы, путем передачи `numeric_only=True`:

In [8]:
titanic.groupby("Sex").mean(numeric_only=True)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Sex,,,,,,,
female,431.028662,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818
male,454.147314,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893


Если нас интересует только средний возраст для каждого пола, выбор столбцов (прямоугольные скобки `[]` как обычно) поддерживается и для сгруппированных данных:

In [9]:
titanic.groupby("Sex")["Age"].mean()

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

In [10]:
# или датафреймом
titanic.groupby("Sex")[["Age"]].mean()

,Age
Sex,
female,27.915709
male,30.726645


![06_groupby_select_detail](https://pandas.pydata.org/docs/_images/06_groupby_select_detail.svg)

Не имеет особого смысла получать среднее значение `Pclass`. Столбец `Pclass` содержит числовые данные, но фактически представляет 3 категории (или факторы) с метками соответственно «1», «2» и «3». Подсчитывать статистику по ним особого смысла нет. Поэтому pandas предоставляет тип данных `Categorical` для обработки этого типа данных. Более подробная информация представлена в разделе Категориальные данные руководства пользователя.

___

⸮ Какова средняя стоимость билета для каждой из комбинаций пола и класса обслуживания?

In [11]:
titanic.groupby(["Sex", "Pclass"])["Fare"].mean()

Sex     Pclass
female  1         106.125798
        2          21.970121
        3          16.118810
male    1          67.226127
        2          19.741782
        3          12.661633
Name: Fare, dtype: float64

In [12]:
# или как DataFrame
titanic.groupby(["Sex", "Pclass"])[["Fare"]].mean()

Fare
Sex    Pclass            
female 1       106.125798
       2        21.970121
       3        16.118810
male   1        67.226127
       2        19.741782
       3        12.661633

Группировка может выполняться по нескольким столбцам одновременно. Укажите имена столбцов в виде списка для метода `groupby()`.

## Подсчитать количество записей по категориям

![06_valuecounts](https://pandas.pydata.org/docs/_images/06_valuecounts.svg)

___

⸮ Каково количество пассажиров в каждом из классов салона?

In [13]:
titanic["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

Метод `value_counts()` подсчитывает количество записей для каждой категории в столбце.

Функция является сокращением, так как на самом деле это групповая операция в сочетании с подсчетом количества записей в каждой группе:

In [14]:
titanic.groupby("Pclass")["Pclass"].count()

Pclass
1    216
2    184
3    491
Name: Pclass, dtype: int64

*Примечание:* И `size`, и `count` могут использоваться в сочетании с `groupby`. В то время как `size` включает значения `NaN` и просто предоставляет количество строк (размер таблицы), `count` исключает пропущенные значения. В методе `value_counts` используйте аргумент `dropna`, чтобы включить или исключить значения `NaN`.

In [15]:
titanic.groupby("Pclass")["Pclass"].size()
# нет нанов

Pclass
1    216
2    184
3    491
Name: Pclass, dtype: int64